In [0]:
# Create widget for pipeline parameter
dbutils.widgets.text("LastRunUTC", "2026-01-01T00:00:00Z")

# Get the value
last_run = dbutils.widgets.get("LastRunUTC")
print(f"Processing data after {last_run}")

In [0]:
spark.conf.set(
  "fs.azure.account.key.projectapisa.dfs.core.windows.net",
  "ACCESSKEY"
)

In [0]:
bronze_path = "abfss://bronze@projectapisa.dfs.core.windows.net/"
df = spark.read.json(bronze_path)
df.show(5)

In [0]:
df = spark.read.json(bronze_path)

In [0]:
df.printSchema()
df.show(5, truncate=False)

In [0]:
from pyspark.sql.functions import col

df.select(
    "id",
    "symbol",
    "name",
    "current_price",
    "market_cap",
    "market_cap_rank",
    "total_volume",
    "ath",
    "ath_date",
    "atl",
    "atl_date",
    "circulating_supply",
    "total_supply"
).show(truncate=False)

In [0]:
from pyspark.sql.functions import struct

df2 = df.withColumn(
    "market_data",
    struct(
        col("current_price"),
        col("market_cap"),
        col("market_cap_rank"),
        col("total_volume"),
        col("ath"),
        col("ath_date"),
        col("atl"),
        col("atl_date"),
        col("circulating_supply"),
        col("total_supply")
    )
)

df2.select("id", "symbol", "market_data").show(truncate=False)

**Note**: DBFS mounting is deprecated and not supported in this environment. 

The bronze container is already accessible using the account key configuration in Cell 2. No mounting is required - continue using the direct path: `abfss://bronze@projectapisa.dfs.core.windows.net/`

In [0]:
# Config
storage_account_name = "projectapisa"
container_name = "bronze"
sas_token = "SASTOKEN"

# Path to folder (all JSONs in bronze container)
bronze_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/"

# Spark configs for SAS
spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account_name}.dfs.core.windows.net",
               "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account_name}.dfs.core.windows.net", sas_token)

# Read all JSON files
df = spark.read.option("multiline", "true").json(bronze_path)

df.printSchema()
df.show(5, truncate=False)

In [0]:
# Count missing values per column
from pyspark.sql.functions import col, count, when

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [0]:
df_clean = df.na.drop()

In [0]:
# Fill nulls with default values for selected columns
df_clean = df.fillna({
    "name": "unknown",
    "symbol": "N/A",
    "current_price": 0,
    "market_cap": 0
})

In [0]:
df_clean = df.na.drop()

In [0]:
# Step 1: Inspect schema
df.printSchema()
df.show(5, truncate=False)

# Step 2: Clean data (fill nulls or drop if needed)
df_clean = df.fillna({
    "name": "unknown",
    "symbol": "N/A",
    "current_price": 0,
    "market_cap": 0
})

# Step 3: Write to Silver Delta
silver_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/silver_delta/"
df_clean.write.format("delta").mode("overwrite").save(silver_path)

# Step 4: Optional - register as Spark SQL table
spark.sql(f"CREATE TABLE IF NOT EXISTS silver_crypto AS SELECT * FROM delta.`{silver_path}`")

# Step 5: Query to verify
spark.sql("SELECT name, symbol, current_price, market_cap FROM silver_crypto LIMIT 10").show()

In [0]:
df_silver = spark.read.format("delta").load(silver_path)

df_silver.show(100, truncate=False)

In [0]:
display(df_silver)

In [0]:
from pyspark.sql.functions import desc

df_gold = df_silver.orderBy(desc("market_cap")).limit(10)

display(df_gold)

In [0]:
from pyspark.sql.functions import col

df_gold_metrics = df_silver.select(
    "name",
    "symbol",
    "current_price",
    "market_cap",
    "total_volume",
    (col("market_cap") / col("total_volume")).alias("cap_to_volume_ratio")
)

display(df_gold_metrics)

In [0]:
container_name = "bronze"  
storage_account_name = "projectapisa"  

gold_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/gold_delta/"

In [0]:
df_gold.write.format("delta").mode("overwrite").saveAsTable("gold_crypto")

In [0]:
spark.sql("SELECT * FROM gold_crypto").show()

In [0]:
df_gold.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold_crypto")

In [0]:
spark.sql("SELECT * FROM gold_crypto").display()

In [0]:
import pandas as pd

# Example: load from CSV if you export your Gold layer
# gold_df = pd.read_csv("gold_layer_real.csv")

# OR if you have it already in memory
gold_data = [
    {
        "name": "Bitcoin", "symbol": "btc", "current_price": 64565, "market_cap": 1290965401652, "high_24h": 67695,
        "low_24h": 63963, "price_change_24h": -2988.26, "price_change_percentage_24h": -4.42, "total_volume": 55325784638
    },
    {
        "name": "WhiteBIT Coin", "symbol": "wbt", "current_price": 48.31, "market_cap": 10321642509, "high_24h": 50.42,
        "low_24h": 47.87, "price_change_24h": -2.01, "price_change_percentage_24h": -3.99, "total_volume": 85229512
    },
    {
        "name": "Stellar", "symbol": "xlm", "current_price": 0.151805, "market_cap": 4990572417, "high_24h": 0.157003,
        "low_24h": 0.14974, "price_change_24h": -0.003, "price_change_percentage_24h": -1.95, "total_volume": 105514419
    }
]

gold_df = pd.DataFrame(gold_data)
gold_df.head()

In [0]:
import plotly.express as px

# Sort by market cap
top_coins = gold_df.sort_values(by='market_cap', ascending=False).head(10)

fig = px.bar(
    top_coins,
    x='name',
    y='market_cap',
    color='name',
    text='market_cap',
    title='Top Coins by Market Cap'
)
fig.update_layout(yaxis_title='Market Cap (USD)', xaxis_title='Coin')
fig.show()

In [0]:
fig2 = px.scatter(
    top_coins,
    x='current_price',
    y='price_change_percentage_24h',
    size='market_cap',
    color='name',
    hover_name='name',
    title='Coin Price vs 24h Change %',
    size_max=60
)
fig2.show()

In [0]:
fig3 = px.bar(
    top_coins,
    x='name',
    y=['low_24h', 'high_24h'],
    title='24h Price Range',
    barmode='group'
)
fig3.show()

In [0]:
fig4 = px.scatter(
    top_coins,
    x='market_cap',
    y='total_volume',
    size='market_cap',
    color='name',
    hover_name='name',
    title='Market Cap vs Total Volume'
)
fig4.show()

In [0]:
# ==========================================
# Crypto Gold Layer Visualization Script
# ==========================================

import pandas as pd
import plotly.express as px

# -------------------------
# 1️⃣ Define your Gold Layer Data
# -------------------------
gold_data = [
    {
        "name": "Bitcoin", "symbol": "btc", "current_price": 64565, "market_cap": 1290965401652, 
        "high_24h": 67695, "low_24h": 63963, "price_change_24h": -2988.26, 
        "price_change_percentage_24h": -4.42, "total_volume": 55325784638
    },
    {
        "name": "WhiteBIT Coin", "symbol": "wbt", "current_price": 48.31, "market_cap": 10321642509, 
        "high_24h": 50.42, "low_24h": 47.87, "price_change_24h": -2.01, 
        "price_change_percentage_24h": -3.99, "total_volume": 85229512
    },
    {
        "name": "Stellar", "symbol": "xlm", "current_price": 0.151805, "market_cap": 4990572417, 
        "high_24h": 0.157003, "low_24h": 0.14974, "price_change_24h": -0.003, 
        "price_change_percentage_24h": -1.95, "total_volume": 105514419
    }
]

# Create DataFrame
gold_df = pd.DataFrame(gold_data)

# -------------------------
# 2️⃣ Top Coins by Market Cap
# -------------------------
top_coins = gold_df.sort_values(by='market_cap', ascending=False).head(10)

fig_marketcap = px.bar(
    top_coins,
    x='name',
    y='market_cap',
    color='name',
    text='market_cap',
    title='Top Coins by Market Cap'
)
fig_marketcap.update_layout(yaxis_title='Market Cap (USD)', xaxis_title='Coin')
fig_marketcap.show()

# -------------------------
# 3️⃣ Price vs 24h Change %
# -------------------------
fig_price_change = px.scatter(
    top_coins,
    x='current_price',
    y='price_change_percentage_24h',
    size='market_cap',
    color='name',
    hover_name='name',
    title='Coin Price vs 24h Change %',
    size_max=60
)
fig_price_change.show()

# -------------------------
# 4️⃣ 24h High / Low Range
# -------------------------
fig_high_low = px.bar(
    top_coins,
    x='name',
    y=['low_24h', 'high_24h'],
    title='24h Price Range',
    barmode='group',
    labels={'value':'Price (USD)', 'variable':'Range'}
)
fig_high_low.show()

# -------------------------
# 5️⃣ Market Cap vs Total Volume
# -------------------------
fig_volume = px.scatter(
    top_coins,
    x='market_cap',
    y='total_volume',
    size='market_cap',
    color='name',
    hover_name='name',
    title='Market Cap vs Total Volume',
    size_max=60
)
fig_volume.show()

# -------------------------
# ✅ Script complete
# -------------------------
print("All visualizations generated successfully!")